In [1]:
# We are going to start importing the libraries we need
# all in one cell. 
# It is a good practice to keep all the imports in one cell so that
# we can easily see what libraries we are using in the notebook.

import pandas as pd


c:\Users\desai\miniforge3\envs\crp4680\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Students and capital improvements
We are going to continue with the datasets that we worked on earlier this week. Again, our objective is to look at the relationship between the **total number of students in a general ed public school** to the **money spent on new school construction and improvements in that school**. 

# 0. Read in data

In [2]:
projects_under_const = pd.read_csv('Active_Projects_Under_Construction.csv')
# Let's pretend we don't have the 'data_year' column, which wasn't in the original dataset anyways
projects_under_const = projects_under_const.drop(columns='data_year')

class_size = pd.read_csv('2021_-_2022_Average_Class_Size_by_School.csv')

## 1.1 Slicing Strings

### 1.1.1 Example 1


The `projects_under_const` has a `Data as Of` column, which gives us some temporal variation in when, at least the data was added to the table. It could be useful, for instance, if we think that `Data as Of` is a rough proxy for when the project was funded or approved. 

In [3]:
projects_under_const.head()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building ID,Building Address,City,Postcode,...,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of
0,NaN,M,2,NaN,0.0,CAP,M777,227 WEST 27TH STREET,Manhattan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAYSIDE HIGH SCHOOL - QUEENS,Q,26,FY19 RESO A AUDITORIUM UPGRADE,1261000.0,CIP,Q405,32-24 CORPORAL KENNEDY STREET,Queens,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/6/22
2,P.S. @ PARCEL F - QUEENS,Q,30,Demo,0.0,CAP,NaN,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/18
3,3K CENTER @ 3893 DYRE AVENUE - BRONX,X,11,Lease,6262000.0,CAP,X501,3893 DYRE AVEUNE,Bronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/4/22
4,P.S. 129 - QUEENS,Q,25,Addition,0.0,CAP,Q129,128-02 7TH AVENUE,Queens,11356.0,...,40.790638,-73.839771,7.0,19.0,945.0,4096774.0,4.039760e+09,Whitestone,"(40.790638, -73.839771)",2/6/19


In [4]:
# Remember that NaN means "Not a Number".
# In other words, it is a missing value
projects_under_const['Data As Of'].head()

0         NaN
1      1/6/22
2    10/30/18
3      8/4/22
4      2/6/19
Name: Data As Of, dtype: object

Let's say we want to extract year from these dates. We have another string-related function we can apply to all of our values under `Data As Of`. 

`.split()` splits strings around given separator/delimiter to create a list of strings. 

Here, we will use `/` as our separator. 

In [5]:
# "str" is a string method that allows us to apply a suite of methods/functions for strings to a column 
projects_under_const['Data As Of'].str.split('/')

0                NaN
1         [1, 6, 22]
2       [10, 30, 18]
3         [8, 4, 22]
4         [2, 6, 19]
            ...     
8996     [11, 2, 22]
8997     [11, 2, 22]
8998     [11, 2, 22]
8999     [11, 2, 22]
9000     [11, 2, 22]
Name: Data As Of, Length: 9001, dtype: object

Now we just have to get the last value (where it exists) and create a new column with the year. 

Here the [-1] is used to get the last element of the list, which we are applying to each element of the column

In [15]:
projects_under_const['Data As Of'].str.split('/').str[-1]

0       NaN
1        22
2        18
3        22
4        19
       ... 
8996     22
8997     22
8998     22
8999     22
9000     22
Name: Data As Of, Length: 9001, dtype: object

Now, let's create a new column called `data_year` with our newly extracted year values. 

In [6]:

projects_under_const['data_year'] = projects_under_const['Data As Of'].str.split('/').str[-1]

In [7]:
# Notice that when there was an NaN, the split function returned a NaN
projects_under_const.head()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building ID,Building Address,City,Postcode,...,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of,data_year
0,NaN,M,2,NaN,0.0,CAP,M777,227 WEST 27TH STREET,Manhattan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAYSIDE HIGH SCHOOL - QUEENS,Q,26,FY19 RESO A AUDITORIUM UPGRADE,1261000.0,CIP,Q405,32-24 CORPORAL KENNEDY STREET,Queens,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/6/22,22
2,P.S. @ PARCEL F - QUEENS,Q,30,Demo,0.0,CAP,NaN,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/18,18
3,3K CENTER @ 3893 DYRE AVENUE - BRONX,X,11,Lease,6262000.0,CAP,X501,3893 DYRE AVEUNE,Bronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/4/22,22
4,P.S. 129 - QUEENS,Q,25,Addition,0.0,CAP,Q129,128-02 7TH AVENUE,Queens,11356.0,...,-73.839771,7.0,19.0,945.0,4096774.0,4.039760e+09,Whitestone,"(40.790638, -73.839771)",2/6/19,19


### 1.1.2 Example 2
We will eventually be comparing school attendance characteristics to money allocated through **merging along a common column name** at the **school level**.

What are our options for merging here? Let's take  look. 

In [8]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15


In [11]:
projects_under_const.head()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building ID,Building Address,City,Postcode,...,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of,data_year
0,NaN,M,2,NaN,0.0,CAP,M777,227 WEST 27TH STREET,Manhattan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAYSIDE HIGH SCHOOL - QUEENS,Q,26,FY19 RESO A AUDITORIUM UPGRADE,1261000.0,CIP,Q405,32-24 CORPORAL KENNEDY STREET,Queens,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/6/22,22
2,P.S. @ PARCEL F - QUEENS,Q,30,Demo,0.0,CAP,NaN,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/18,18
3,3K CENTER @ 3893 DYRE AVENUE - BRONX,X,11,Lease,6262000.0,CAP,X501,3893 DYRE AVEUNE,Bronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/4/22,22
4,P.S. 129 - QUEENS,Q,25,Addition,0.0,CAP,Q129,128-02 7TH AVENUE,Queens,11356.0,...,-73.839771,7.0,19.0,945.0,4096774.0,4.039760e+09,Whitestone,"(40.790638, -73.839771)",2/6/19,19


Even though there is a **School Name** column in both datasets, the format seems to be quite different. 
- For the `projects_under_const` dataset, the school names are all over the place. Some are the name and borough separated by a `-`, some also include an `@` followed by a rough locationn. 
- For the `class_size` df, the school names are consistent, but we can see that it might be a pain to match the two. 

In [12]:
projects_under_const['School Name']

0                                        NaN
1               BAYSIDE HIGH SCHOOL - QUEENS
2                   P.S. @ PARCEL F - QUEENS
3       3K CENTER @ 3893 DYRE AVENUE - BRONX
4                          P.S. 129 - QUEENS
                        ...                 
8996                     P.S. 236 - BROOKLYN
8997                        P.S. 277 - BRONX
8998                       P.S. 5 - BROOKLYN
8999                        P.S. 182 - BRONX
9000                        I.S. 127 - BRONX
Name: School Name, Length: 9001, dtype: object

In [13]:
class_size['School Name']

0                              PS 015 ROBERTO CLEMENTE
1                              PS 015 ROBERTO CLEMENTE
2                              PS 015 ROBERTO CLEMENTE
3                              PS 015 ROBERTO CLEMENTE
4                              PS 015 ROBERTO CLEMENTE
                             ...                      
12440                  PS 377 ALEJANDRINA B DE GAUTIER
12441                        JHS 383 PHILIPPA SCHUYLER
12442                        JHS 383 PHILIPPA SCHUYLER
12443                      PS /IS 384 FRANCES E CARTER
12444    EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION
Name: School Name, Length: 12445, dtype: object

Instead, I noticed that there's a `Building ID` column in the `projects_under_constr` DF (dataframe, for short) that, though is described unhelpfully as "ID of the Building" in the documentation, looks to be similar to the `DBN` from `class_size` DF. 


In fact, when I look at what `DBN` is in the class size documentation, it says that this column "Denotes cocatenation[sic] of district, borough and three digit school number."

I'm going to guess here that if I extract the "borough and three digit school number" part of `DBN`, this will match my `Building ID` column. 

Thankfully, it seems like there is a fixed number of characters I need extract from `DBN`: 
- Borough = 1
- School number = 3

In total, I will need the last 4 characters from `DBN`. We'll do this again with a string splice. 

In [14]:
# Here I am going to use the str method to get the last 4 characters of the DBN
# within the square brackets, I am taking everything fourth from the end onwards
# That's what -4 means

class_size['DBN'].str[-4:]

0        M015
1        M015
2        M015
3        M015
4        M015
         ... 
12440    K377
12441    K383
12442    K383
12443    K384
12444    K562
Name: DBN, Length: 12445, dtype: object

Quick review of selecting ranges:

In [15]:
# It's a little strange because backwards counting starts at -1
class_size['DBN'].str[-1:]

0        5
1        5
2        5
3        5
4        5
        ..
12440    7
12441    3
12442    3
12443    4
12444    2
Name: DBN, Length: 12445, dtype: object

In [16]:
## Here, 4: means that I want to start at the fifth character 
## because python starts counting at 0 for forward counting
class_size['DBN'].str[4:]


0        15
1        15
2        15
3        15
4        15
         ..
12440    77
12441    83
12442    83
12443    84
12444    62
Name: DBN, Length: 12445, dtype: object

In [17]:
## And if I wanted to select a slice of the string in the middle
## I can do the following
class_size['DBN'].str[1:4]

0        1M0
1        1M0
2        1M0
3        1M0
4        1M0
        ... 
12440    2K3
12441    2K3
12442    2K3
12443    2K3
12444    2K5
Name: DBN, Length: 12445, dtype: object

Back to our exericse, let's assign our slice to a new colunn called `bid`

In [18]:
class_size['bid'] = class_size['DBN'].str[-4:]

In [19]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,bid
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15,M015
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,M015
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15,M015
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,M015
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15,M015


## 1.3 Aggregating data: Split-apply-combine
The split-apply-combine operation that is very common in pandas. We often want to aggregate data by some category. For example, we might want to know the total amount of construction money allocated by school. Or we might want to know the total number of students in each school.

For the projects under construction, let's group by the `Building ID`, which is our index for school here. and sum all the award amounts by school to get the: 
- Total construction award amount per school

In [20]:
projects_under_const.groupby('Building ID').count()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building Address,City,Postcode,Borough,...,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of,data_year
Building ID,,,,,,,,,,,,,,,,,,,,,
K001,11,11,11,11,11,11,11,11,10,11,...,10,10,10,10,10,10,10,10,11,11
K002,9,9,9,9,9,9,9,9,8,9,...,8,8,8,8,8,8,8,8,9,9
K003,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,3,3
K005,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
K007,9,9,9,9,9,9,9,9,7,9,...,7,7,7,7,7,7,7,7,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X843,17,17,17,17,17,17,17,17,15,17,...,15,15,15,15,15,15,15,15,16,16
X862,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
X930,5,5,5,5,5,5,5,5,4,5,...,4,4,4,4,4,4,4,4,5,5


In [21]:
## Remember that .sum() will only sum the numeric columns
projects_under_const.groupby('Building ID').sum()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building Address,City,Postcode,Borough,...,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of,data_year
Building ID,,,,,,,,,,,,,,,,,,,,,
K001,P.S. 1 - BROOKLYNP.S. 1 - BROOKLYNP.S. 1 - BRO...,KKKKKKKKKKK,165,FY18 RESO A IP SURVEILLANCE CAMERA INSTALLATIO...,4.409425e+06,CIPCIPCIPCIPCIPCIPCIPCIPCIPCIPCIP,309 47 STREET309 47 STREET309 47 STREET309 47 ...,BrooklynBrooklynBrooklynBrooklynBrooklynBrookl...,108098.0,BROOKLYNBROOKLYNBROOKLYNBROOKLYNBROOKLYNBROOKL...,...,-740.122890,2470.0,349.0,800.0,30118210.0,3.007550e+10,Sunset Park WestSunset Park WestSunset Park We...,"(40.649042, -74.012289)(40.649042, -74.012289)...",10/30/182/6/1911/1/1911/1/192/3/202/3/208/9/21...,1819191920202121222222
K002,I.S. 2 - BROOKLYNI.S. 2 - BROOKLYNI.S. 2 - BRO...,KKKKKKKKK,153,FY19 RESO A LIBRARY UPGRADEFY19 RESO A LIBRARY...,6.349880e+07,CIPCIPCIPCIPCIPCIPCIPCIPCIP,655 PARKSIDE AVENUE655 PARKSIDE AVENUE655 PARK...,BrooklynBrooklynBrooklynBrooklynBrooklynBrookl...,85532.0,BROOKLYNBROOKLYNBROOKLYNBROOKLYNKKKBROOKLYNBRO...,...,-591.612664,2472.0,320.0,6416.0,27025608.0,2.440400e+10,Prospect Lefferts Gardens-WingateProspect Leff...,"(40.656183, -73.951583)(40.656183, -73.951583)...",11/1/192/3/208/9/2111/9/211/6/228/4/228/4/2211...,192021212222222222
K003,P.S. 3 - BROOKLYNP.S. 3 - BROOKLYNP.S. 3 - BRO...,KKKK,52,PLANYC BOILER CNVRSN/PLANYC CLIMATE CTRLELECTR...,2.693780e+07,CIPCIPCIPCIP,50 JEFFERSON AVENUE50 JEFFERSON AVENUE50 JEFFE...,BrooklynBrooklynBrooklynBrooklyn,43962.0,BROOKLYN KKBROOKLYN,...,-295.821352,912.0,144.0,908.0,12229624.0,1.207992e+10,Clinton Hill ...,"(40.682663, -73.955338)(40.682663, -73.955338)...",5/5/198/4/2211/2/22,192222
K005,P.S. 5 - BROOKLYNP.S. 5 - BROOKLYNP.S. 5 - BRO...,KKKK,64,FY19 RESO A AUDITORIUM UPGRADEFY19 RESO A AUDI...,1.700470e+07,CIPCIPCIPCIP,820 HANCOCK STREET820 HANCOCK STREET820 HANCOC...,BrooklynBrooklynBrooklynBrooklyn,43238.0,KBROOKLYNBROOKLYNBROOKLYN,...,-295.689852,1212.0,164.0,1508.0,12159624.0,1.205960e+10,Stuyvesant HeightsStuyvesant HeightsStuyvesant...,"(40.685695, -73.922463)(40.685695, -73.922463)...",5/5/1911/1/192/3/2011/2/22,19192022
K007,P.S. 7 - BROOKLYNP.S. 7 - BROOKLYNP.S. 7 - BRO...,KKKKKKKKK,171,LSP- VACUUM CONDENSATE PUMPSTCU REMOVAL/PLAYGR...,2.768991e+06,CIPCIPCIPCIPCIPCIPCIPCIPCIP,858 JAMAICA AVENUE858 JAMAICA AVENUE858 JAMAIC...,BrooklynBrooklynBrooklynBrooklynBrooklynBrookl...,74410.0,KKKBROOKLYNBROOKLYNBROOKLYNBROOKLYNBROOKLYNBRO...,...,-517.111763,2135.0,259.0,827407.0,21636986.0,2.128742e+10,Cypress Hills-City LineCypress Hills-City Line...,"(40.689697, -73.873109)(40.689697, -73.873109)...",1/6/221/6/225/5/1911/1/192/3/208/9/218/9/2111/...,222219192021212121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X843,P.S. 246 - BRONXP.S. 246 - BRONXP.S. 246 - BRO...,XXXXXXXXXXXXXXXXX,170,ELEVATOR/EXT MSNRY/FLOOD/ PARAPETS/ROOFS/WALLE...,1.272268e+08,CIPCIPCIPCIPCIPCIPCIPCIPCIPCIPCIPCIPCIPCIPCIPC...,2641 GRAND CONCOURSE2641 GRAND CONCOURSE2641 G...,BronxBronxBronxBronxBronxBronxBronxBronxBronxB...,156664.0,BRONX BRONX BRONX BRONXBRONXXXXBRONXB...,...,-1108.421370,2105.0,203.0,6015.0,30207450.0,3.047520e+10,Bedford Park-Fordham North ...,"(40.865593, -73.894758)(40.865593, -73.894758)...",1/31/1811/13/1710/30/182/6/191/6/225/5/195/5/1...,18171819221919191920202121212122
X862,MOTT HALL CHARTER SCHOOLMOTT HALL CHARTER SCHO...,XXX,27,LOW VOLTAGE ELECTRICAL SYSTEMLOW VOLTAGE ELECT...,3.513000e+06,CIPCIPCIP,1260 FRANKLIN AVENUE1260 FRANKLIN AVENUE1260 F...,BronxBronxBronx,31368.0,BRONX BRONXBRONX,...,-221.708115,9.0,35.0,447.0,6012906.0,6.078450e+09,Morrisania-Melrose ...,"(40.830995, -73.902705)(40.830995, -73.902705)...",1/31/1810/30/182/6/19,181819
X930,P.S. 9 - BRONXP.S. 9 - BRONXP.S. 9 - BRONXP.S....,XXXXX,50,CRACKED MASONRY / PARAPETS / ROOF / ROOFSROOFC...,8.898153e+07,CIPCIPCIPCIPCIP,230 EAST 183 STREET230 E

In [23]:
projects_under_const.columns

Index(['School Name', 'BoroughCode', 'Geographical District',
       'Project Description', 'Construction Award', 'Project type',
       'Building ID', 'Building Address', 'City', 'Postcode', 'Borough',
       'Latitude', 'Longitude', 'Community Board', 'Council District',
       'Census Tract', 'BIN', 'BBL', 'NTA', 'Location 1', 'Data As Of',
       'data_year'],
      dtype='object')

Most of these columns are gibberish after we sum (for ex: we don't need a sum of latitudes and longitudes by school). Let's just select the columns we want to use: 

In [24]:
# Remember the brackets after a DF allow you to select columns
projects_under_const.groupby('Building ID').sum()['Construction Award']

Building ID
K001    4.409425e+06
K002    6.349880e+07
K003    2.693780e+07
K005    1.700470e+07
K007    2.768991e+06
            ...     
X843    1.272268e+08
X862    3.513000e+06
X930    8.898153e+07
X970    9.079000e+06
X973    3.092000e+05
Name: Construction Award, Length: 1181, dtype: float64

Let's assign this to a new variable name. 

In [25]:
projects_under_const_agg = projects_under_const.groupby('Building ID').sum()['Construction Award']

Here you can see that the result is a **pandas Series**. To make this easier to work with during the merge, let's transform this into a pandas DF. 

I'm going to use a function call `.reset_index()` as a trick to do this. `.reset_index()` is a method that resets the index of a dataframe to a column of your choice. The default is to reset the index to a column of sequential numbers

In [26]:
# See how Building ID, which was the index before, is now a column. 
# and the index is i just 0,...,1180

projects_under_const_agg.reset_index()

,Building ID,Construction Award
0,K001,4.409425e+06
1,K002,6.349880e+07
2,K003,2.693780e+07
3,K005,1.700470e+07
4,K007,2.768991e+06
...,...,...
1176,X843,1.272268e+08
1177,X862,3.513000e+06
1178,X930,8.898153e+07
1179,X970,9.079000e+06


In [28]:
projects_under_const_agg = projects_under_const_agg.reset_index()

In [29]:
projects_under_const_agg

,Building ID,Construction Award
0,K001,4.409425e+06
1,K002,6.349880e+07
2,K003,2.693780e+07
3,K005,1.700470e+07
4,K007,2.768991e+06
...,...,...
1176,X843,1.272268e+08
1177,X862,3.513000e+06
1178,X930,8.898153e+07
1179,X970,9.079000e+06


Let's do something similar with the `class_size` df. As we can see from the below, our data is likely one row per grade and program. We want to aggregate this to the school level. 

In [30]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,bid
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15,M015
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,M015
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15,M015
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,M015
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15,M015


I'm first going to filter my DF since I just want 'Gen Ed' in order not to skew the representative class size by special programs. 

In [31]:
# .unique() returns a list of all the unique values in a column
class_size['Program Type'].unique()

array(['G&T', 'ICT', 'Gen Ed', 'ICT & G&T', 'SC 12:1:1', 'SC 12:1',
       'SC 8:1:1', 'SC 12:1:4', 'SC 6:1:1', 'SC 15:1'], dtype=object)

In [32]:
# I am going to use the == operator to check if the value in the Program Type column is equal to 'Gen Ed'
# Then we'll set this filtered dataframe to a new variable
# and use that new dataframe from now on. 
class_size_new = class_size[class_size['Program Type']=='Gen Ed']

In [33]:
class_size_new.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,bid
18,01M020,PS 020 ANNA SILVER,K,Gen Ed,33,2,16.5,<15,20,M020
20,01M020,PS 020 ANNA SILVER,1,Gen Ed,31,2,15.5,15,16,M020
22,01M020,PS 020 ANNA SILVER,2,Gen Ed,24,2,12.0,<15,15,M020
24,01M020,PS 020 ANNA SILVER,3,Gen Ed,30,3,10.0,<15,<15,M020
26,01M020,PS 020 ANNA SILVER,4,Gen Ed,18,2,9.0,<15,<15,M020


Now let's groupby `bid` and sum all the grades within each school. 

In [34]:
class_size_new.groupby('bid').sum()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size
bid,,,,,,,,,
K001,15K00115K00115K00115K00115K00115K001,PS 001 THE BERGENPS 001 THE BERGENPS 001 THE B...,K12345,Gen EdGen EdGen EdGen EdGen EdGen Ed,492,25,117.4,151517201919,191921262323
K002,17K00217K00217K002,PARKSIDE PREPARATORY ACADEMYPARKSIDE PREPARATO...,678,Gen EdGen EdGen Ed,226,10,66.7,<152417,212826
K003,13K00313K00313K00313K00313K00313K003,PS 003 THE BEDFORD VILLAGEPS 003 THE BEDFORD V...,K12345,Gen EdGen EdGen EdGen EdGen EdGen Ed,201,12,100.5,<15<1515191916,17<1515212017
K005,16K00516K00516K00516K005,PS 005 DR RONALD MCNAIRPS 005 DR RONALD MCNAIR...,K123,Gen EdGen EdGen EdGen Ed,70,4,70.0,<15151926,<15151926
K006,17K00617K00617K00617K00617K00617K006,PS 006PS 006PS 006PS 006PS 006PS 006,K12345,Gen EdGen EdGen EdGen EdGen EdGen Ed,347,19,111.1,161718191517,212019211922
...,...,...,...,...,...,...,...,...,...
X584,07X58407X58407X584,IS 584IS 584IS 584,678,Gen EdGen EdGen Ed,118,6,59.0,<152023,172127
X593,09X59309X59309X593,SOUTH BRONX INTERNATIONAL MSSOUTH BRONX INTERN...,678,Gen EdGen EdGen Ed,63,3,63.0,201924,201924
X594,09X59409X59409X594,MS 594MS 594MS 594,678,Gen EdGen EdGen Ed,104,5,60.0,161725,161927


Again, we'll just need the `Number of Students` column here. And I'm going to do the `reset_index()` trick again. This time, I'm going to string all these steps together

In [35]:
# Pandas reads this code from left to right and will apply each function on the right to the everything on the left
# So, first we are going to group by bid
# Then we are going to sum each group
# Then from the entire summed dataframe, we are going to select the total_students_in_grade column
# Selecting that series, we are going to reset the index to create our new dataframe. .

class_size_new_agg = class_size_new.groupby('bid').sum()['Number of Students'].reset_index()

In [36]:
class_size_new_agg.head()

,bid,Number of Students
0,K001,492
1,K002,226
2,K003,201
3,K005,70
4,K006,347


## 1.4 Merging dataframes
Lastly, we want to do the merge part using the `.merge()` function.

It follows this format: 
```
df1.merge(df2,left_on='df1 col for merging', right_on='df2 col for merge')
```

By default, the type of merge will be **inner**, however, here are other types: 
- ‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’

 We are going to merge 
- `projects_under_cont_agg`
- `class_size_new_agg`

In [37]:
## Here, I'm doing an inner merge, which means that I am only going to keep the rows that have a match in both dataframes
## This means only the schools that received a construction award are going to be kept
class_size_new_agg.merge(projects_under_const_agg,
                         left_on='bid',
                         right_on='Building ID')

,bid,Number of Students,Building ID,Construction Award
0,K001,492,K001,4409425.0
1,K002,226,K002,63498798.0
2,K003,201,K003,26937800.0
3,K005,70,K005,17004700.0
4,K007,424,K007,2768991.0
...,...,...,...,...
658,X448,178,X448,186680000.0
659,X449,163,X449,41250000.0
660,X468,175,X468,172599000.0
661,X481,146,X481,43055000.0


In [38]:
merged_df = class_size_new_agg.merge(projects_under_const_agg,
                         left_on='bid',
                         right_on='Building ID', 
                         how='left')

Ok, finally, to get to our answer, we're going to apply the `.corr()` function to our dataframe. The [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) tells us that this function computes pairwise correlation of columns, excluding NA/null values.

The default method is a 'Pearson' correlation, with all methods being: 
- pearson : standard correlation coefficient
- kendall : Kendall Tau correlation coefficient
- spearman : Spearman rank correlation

In [42]:
# Yikes, 0.154657 correlation. I guess I assumed wrong that there would be an strong correlation between the number of students in a school and the amount of money spent on construction.
merged_df[['Number of Students', 'Construction Award']].corr()

,Number of Students,Construction Award
Number of Students,1.000000,0.154657
Construction Award,0.154657,1.000000


Well, that wasn't the strong relationship I expected. This doesn't necessarily mean there's no relationship, but I'm going to stop my investigation here for now. 

### 1.4.1 Merging with pd.concat

In [43]:
data_q1 = {
    'Product_ID': ['P001', 'P002', 'P003', 'P004'],
    'Q1_Sales': [250, 150, 200, 300]
}
df_q1 = pd.DataFrame(data_q1)

data_q2 = {
    'Product_ID': ['P001', 'P002', 'P003', 'P004'],
    'Q2_Sales': [260, 110, 210, 310]
}
df_q2 = pd.DataFrame(data_q2)

data_q3 = {
    'Product_ID': ['P001', 'P002', 'P003', 'P004'],
    'Q3_Sales': [270, 120, 220, 320]
}
df_q3 = pd.DataFrame(data_q3)


In [44]:
df_q1

,Product_ID,Q1_Sales
0,P001,250
1,P002,150
2,P003,200
3,P004,300


In [45]:
df_q2

,Product_ID,Q2_Sales
0,P001,260
1,P002,110
2,P003,210
3,P004,310


In [46]:
df_q3

,Product_ID,Q3_Sales
0,P001,270
1,P002,120
2,P003,220
3,P004,320


In [47]:
pd.concat([df_q1,df_q2,df_q3])          

,Product_ID,Q1_Sales,Q2_Sales,Q3_Sales
0,P001,250.0,NaN,NaN
1,P002,150.0,NaN,NaN
2,P003,200.0,NaN,NaN
3,P004,300.0,NaN,NaN
0,P001,NaN,260.0,NaN
1,P002,NaN,110.0,NaN
2,P003,NaN,210.0,NaN
3,P004,NaN,310.0,NaN
0,P001,NaN,NaN,270.0
1,P002,NaN,NaN,120.0


In [48]:
pd.concat([df_q1,df_q2,df_q3],axis=0)          

,Product_ID,Q1_Sales,Q2_Sales,Q3_Sales
0,P001,250.0,NaN,NaN
1,P002,150.0,NaN,NaN
2,P003,200.0,NaN,NaN
3,P004,300.0,NaN,NaN
0,P001,NaN,260.0,NaN
1,P002,NaN,110.0,NaN
2,P003,NaN,210.0,NaN
3,P004,NaN,310.0,NaN
0,P001,NaN,NaN,270.0
1,P002,NaN,NaN,120.0


In [49]:
pd.concat([df_q1,df_q2,df_q3],axis=1)   

,Product_ID,Q1_Sales,Product_ID,Q2_Sales,Product_ID,Q3_Sales
0,P001,250,P001,260,P001,270
1,P002,150,P002,110,P002,120
2,P003,200,P003,210,P003,220
3,P004,300,P004,310,P004,320


## 1.5 Parsing dates
There is a datetime data type in Pandas that allows us to turn columns with dates and date-times into a `datetime` type. It uses the function 

`pd.to_datetime(df['datetime column])`

Let's try that: 

In [50]:
projects_under_const['Data As Of']

0            NaN
1         1/6/22
2       10/30/18
3         8/4/22
4         2/6/19
          ...   
8996     11/2/22
8997     11/2/22
8998     11/2/22
8999     11/2/22
9000     11/2/22
Name: Data As Of, Length: 9001, dtype: object

In [51]:
pd.to_datetime(projects_under_const['Data As Of'])

C:\Users\desai\AppData\Local\Temp\ipykernel_13784\4083214181.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(projects_under_const['Data As Of'])


0             NaT
1      2022-01-06
2      2018-10-30
3      2022-08-04
4      2019-02-06
          ...    
8996   2022-11-02
8997   2022-11-02
8998   2022-11-02
8999   2022-11-02
9000   2022-11-02
Name: Data As Of, Length: 9001, dtype: datetime64[ns]

Pretty easy! In the background, Pandas is inferring what your date-time format is. You can also state this more explicitly: 

In [52]:
## Lower-case "y" means the year is represented by the last two digits
## Upper-case "Y" means the year is represented by the entire year
## So, if we have 2021, we should use "Y"
## If we have 21, we should use "y"

pd.to_datetime(projects_under_const['Data As Of'], format='%m/%d/%y')

0             NaT
1      2022-01-06
2      2018-10-30
3      2022-08-04
4      2019-02-06
          ...    
8996   2022-11-02
8997   2022-11-02
8998   2022-11-02
8999   2022-11-02
9000   2022-11-02
Name: Data As Of, Length: 9001, dtype: datetime64[ns]

Now we can create a new column from this.

In [53]:
projects_under_const['data_date_new'] = pd.to_datetime(projects_under_const['Data As Of'], format='%m/%d/%y')

Now we can access date characteristics. 

In [54]:
projects_under_const['data_date_new'].dt.year

0          NaN
1       2022.0
2       2018.0
3       2022.0
4       2019.0
         ...  
8996    2022.0
8997    2022.0
8998    2022.0
8999    2022.0
9000    2022.0
Name: data_date_new, Length: 9001, dtype: float64

In [55]:
projects_under_const['data_date_new'].dt.month

0        NaN
1        1.0
2       10.0
3        8.0
4        2.0
        ... 
8996    11.0
8997    11.0
8998    11.0
8999    11.0
9000    11.0
Name: data_date_new, Length: 9001, dtype: float64

# In-Class Exercise 
Using the `FDNY_Firehouse_Listing.csv` dataset, show which neighborhoods (NTAs) have the most Firehouses. 

In [69]:
## INSERT YOUR CODE HERE
df_firehouse = pd.DataFrame(pd.read_csv("FDNY_Firehouse_Listing.csv"))

In [70]:
# there is no missing values in the column for Facility Name, so I will use this to count the number of entries in each group
df_firehouse['FacilityName'].isna().sum()

0

In [84]:
#group the dataset by neighborhood, and I counted the number of entries in FacilityName (because there are no missing values)
grouped_firehouses_by_neighborhood = df_firehouse.groupby("NTA").count()
grouped_firehouses_by_neighborhood['FacilityName'].sort_values(ascending=False)

NTA
SoHo-TriBeCa-Civic Center-Little Italy    5
Midtown-Midtown South                     5
Flatbush                                  3
North Side-South Side                     3
Ocean Hill                                3
                                         ..
Fresh Meadows-Utopia                      1
Fort Greene                               1
Forest Hills                              1
Fordham South                             1
park-cemetery-etc-Queens                  1
Name: FacilityName, Length: 146, dtype: int64

In [99]:
#I can get the max number of fire stations in neighborhood, which is 5, and find those neighborhoods

firehouse_count = grouped_firehouses_by_neighborhood['FacilityName'].sort_values(ascending=False).reset_index()

#also rename the column to be descriptive (it's the number of firehouses, not facility name)
firehouse_count = firehouse_count.rename(columns={"FacilityName": "Number of Firehouses"})

max_firestation_per_neighborhood = firehouse_count['Number of Firehouses'].max()

# display the neighborhoods with the most number of firehouses
firehouse_count[firehouse_count['Number of Firehouses']==max_firestation_per_neighborhood]

,NTA,Number of Firehouses
0,SoHo-TriBeCa-Civic Center-Little Italy,5
1,Midtown-Midtown South,5
